In [ ]:
import sys
import imaplib
import getpass
import email
import email.header
import datetime
import pandas as pd
import numpy as np


EMAIL_ACCOUNT = "jarokah528@abbuzz.com"


EMAIL_FOLDER = "INBOX"


def process_mailbox(M):
    """
    Do something with emails messages in the folder.  
    For the sake of this example, print some headers.
    """

    rv, data = M.search(None, "ALL")
    if rv != 'OK':
        print("No messages found!")
        return
    #data = pd.DataFrame('Num','Subject','Category')
    numlst = []
    sublst = []
    joblst = []
    for num in data[0].split():
        rv, data = M.fetch(num, '(RFC822)')
        if rv != 'OK':
            print("ERROR getting message", num)
            return

        msg = email.message_from_bytes(data[0][1])
        hdr = email.header.make_header(email.header.decode_header(msg['Thank you for applying']))
        subject = str(hdr)
        numlst.append(num)
        sublst .append(subject)
        joblst.append("Job")
#     df = pd.DataFrame(numlst,columns=['Number','Subject','Category'])
        df = {'Number': numlst,
        'Subject': sublst, 'Category': ['Job']}
        df = pd.DataFrame(df)
        df.to_csv("output.csv")
        print('Message %s: %s' % (num, subject))
        
        print('Raw Date:', msg['Date'])


M = imaplib.IMAP4_SSL('imap.gmail.com')

try:
    rv, data = M.login(EMAIL_ACCOUNT, getpass.getpass())
except imaplib.IMAP4.error:
    print ("LOGIN FAILED!!! ")
    sys.exit(1)

print(rv, data)

rv, mailboxes = M.list()
if rv == 'OK':
    print("Mailboxes:")
    print(mailboxes)

rv, data = M.select(EMAIL_FOLDER)
if rv == 'OK':
    print("Processing mailbox...\n")
    process_mailbox(M)
    M.close()
else:
    print("ERROR: Unable to open mailbox ", rv)

M.logout()